In [ ]:
!pip install tensorflow-probability

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 1.8 MB/s eta 0:00:00a 0:00:010m


In [6]:
import tensorflow as tf
import tensorflow_probability as tfp
import pandas as pd

In [19]:
tfp.distributions.Binomial(total_count=2, probs=1 - 0.5).prob(1)

NameError: name 'tfp' is not defined

In [17]:

def calculate_probabilities_tf(C_TR, C_TA, C_NR, C_NA, major, minor, totalCN, TiN):
    C_T = C_TA + C_TR
    C_N = C_NA + C_NR
    C = C_T + C_N
    C_A = C_TA + C_NA
    
    if C == 0:
        return tf.constant(0.0, dtype=tf.float32)

    # Calculate likelihoods
    P_R_no_mutation = tfp.distributions.Binomial(total_count=C, probs=1 - TNR).prob(C_A)
    P_R_homozygous = tfp.distributions.Binomial(total_count=C, probs=TPR).prob(C_A)
    P_R_heterozygous = tfp.distributions.Binomial(total_count=C, probs=(TPR + (1 - TNR)) / 2).prob(C_A)
    P_R_clonal = tf.zeros_like(C_A, dtype=tf.float32)

    # Check for totalCN = 0
    if totalCN == 0:
        P_R_clonal = tfp.distributions.Binomial(total_count=C, probs=1 - TNR).prob(C_A)
    else:
        VAF_values = tf.range(minor, major + 1, dtype=tf.float32) / totalCN
        
        # Use vectorized operations
        tumor_prob = TiT * (VAF_values[:, tf.newaxis] * TPR + (1 - VAF_values[:, tf.newaxis]) * (1 - TNR)) + (1 - TiT) * (1 - TNR)
        normal_prob = TiN * (VAF_values[:, tf.newaxis] * TPR + (1 - VAF_values[:, tf.newaxis]) * (1 - TNR)) + (1 - TiN) * (1 - TNR)

        tumor_result = tfp.distributions.Binomial(total_count=C_T, probs=tumor_prob).prob(C_TA)
        normal_result = tfp.distributions.Binomial(total_count=C_N, probs=normal_prob).prob(C_NA)

        # Average over VAF values
        P_R_clonal = tf.reduce_mean(tumor_result * normal_result, axis=0)

    # Combine the probabilities
    P_R = (P_R_no_mutation * p_no_mutation + 
            P_R_homozygous * p_homozygous + 
            P_R_heterozygous * p_heterozygous + 
            P_R_clonal * p_clonal)

    return -tf.math.log(P_R)

def total_log_likelihood(TiN, dataset):
    total_log_likelihood = tf.constant(0.0, dtype=tf.float32)

    # Iterate over batches of data for memory efficiency
    for batch in dataset:
        total_log_likelihood += calculate_probabilities_tf(
            batch['tumor_ref'], batch['tumor_alt'], batch['normal_ref'], batch['normal_alt'],
            batch['major'], batch['minor'], batch['totalCN'], TiN
        )
    return total_log_likelihood

In [11]:
# Load your data with Pandas or Dask (for large files) and convert it to TensorFlow Dataset
df = pd.read_csv("/rsrch8/home/genetics/tchu/TCGA_LUAD/estimate/head21.txt", sep = "\t", header = None, 
names=["chr", "position", "major", "minor", "totalCN", "tumor_ref", "tumor_alt", "normal_ref", "normal_alt"],
dtype={'chr': int, 'position': int, 'major': int, 'minor': int, 'totalCN': int, 'tumor_ref': int, 'tumor_alt': int, 'normal_ref': int, 'normal_alt': int},
index_col=False)  # Load your large dataset
df.head()

,chr,position,major,minor,totalCN,tumor_ref,tumor_alt,normal_ref,normal_alt
0,21,13507816,1,1,2,103,0,22,0
1,21,13507817,1,1,2,103,0,22,0
2,21,13507818,1,1,2,102,0,21,0
3,21,13507819,1,1,2,102,0,21,0
4,21,13507820,1,1,2,101,0,21,0


In [12]:
dataset = tf.data.Dataset.from_tensor_slices({
    'tumor_ref': tf.constant(df['tumor_ref'].values, dtype=tf.float32),
    'tumor_alt': tf.constant(df['tumor_alt'].values, dtype=tf.float32),
    'normal_ref': tf.constant(df['normal_ref'].values, dtype=tf.float32),
    'normal_alt': tf.constant(df['normal_alt'].values, dtype=tf.float32),
    'major': tf.constant(df['major'].values, dtype=tf.float32),
    'minor': tf.constant(df['minor'].values, dtype=tf.float32),
    'totalCN': tf.constant(df['totalCN'].values, dtype=tf.float32),
})
dataset = dataset.batch(1000)  # Adjust batch size based on available memory

In [18]:
TiN = tf.Variable(0.1, dtype=tf.float32)  # Starting guess for TiN
TPR = 0.95  # Assumed true positive rate
TNR = 0.95  # Assumed true negative rate
TiT = 0.26   # Assumed tumor-in-tumor contamination

# Define the optimizer
optimizer = tf.optimizers.Adam(learning_rate=0.01)

# Training loop to minimize negative log-likelihood
for epoch in range(100):  # Number of optimization steps
    with tf.GradientTape() as tape:
        loss = total_log_likelihood(TiN, dataset)
    gradients = tape.gradient(loss, [TiN])
    optimizer.apply_gradients(zip(gradients, [TiN]))

    # Print progress
    print(f"Epoch {epoch}, Log Likelihood: {-loss.numpy()}, TiN: {TiN.numpy()}")

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()